Bộ dữ liệu MachineDowntime.csv chứa thông tin về các máy móc trong dây chuyền sản xuất và các yếu tố ảnh hưởng đến hoạt động của chúng. Dưới đây là các thuộc tính trong bộ dữ liệu:

* Date: Dữ liệu dạng chuỗi, ngày của sự kiện.
* Machine_ID: Dữ liệu dạng chuỗi, mã định danh của máy móc.
* Assembly_Line_No: Dữ liệu dạng chuỗi, số dây chuyền lắp ráp.
* Hydraulic_Pressure(bar): Áp suất thủy lực, kiểu số thực.
* Coolant_Pressure(bar): Áp suất chất làm mát, kiểu số thực.
* Air_System_Pressure(bar): Áp suất hệ thống khí, kiểu số thực.
* Coolant_Temperature: Nhiệt độ chất làm mát, kiểu số thực.
* Hydraulic_Oil_Temperature(?C): Nhiệt độ dầu thủy lực, kiểu số thực.
* Spindle_Bearing_Temperature(?C): Nhiệt độ vòng bi trục chính, kiểu số thực.
* Spindle_Vibration(?m): Độ rung trục chính, kiểu số thực.
* Tool_Vibration(?m): Độ rung của công cụ, kiểu số thực.
* Spindle_Speed(RPM): Tốc độ quay trục chính (số vòng/phút), kiểu số thực.
* Voltage(volts): Điện áp, kiểu số thực.
* Torque(Nm): Mô-men xoắn, kiểu số thực.
* Cutting(kN): Lực cắt, kiểu số thực.
* Downtime: Dữ liệu dạng chuỗi, chỉ định thời gian ngừng hoạt động của máy.

Bộ dữ liệu này quản lý thông tin về hoạt động và hiệu suất của máy móc trong dây chuyền sản xuất, nhằm đánh giá và dự đoán các yếu tố kỹ thuật có ảnh hưởng đến năng suất.


Mô tả bài toán code và mục tiêu

* Bài toán ở đây là bài toán hồi quy, nhằm dự đoán tốc độ quay trục chính (Spindle_Speed(RPM)) dựa trên các yếu tố đầu vào khác như áp suất, nhiệt độ, độ rung, điện áp và mô-men xoắn. Mục tiêu là xây dựng một mô hình hồi quy tuyến tính sử dụng bộ học SGDRegressor để dự đoán tốc độ này và đánh giá mô hình bằng cách tính toán lỗi bình phương trung bình (MSE) và hệ số xác định (R²).

In [2]:
!pip install scikit-learn
from sklearn.linear_model import SGDRegressor

Bộ học hồi quy sử dụng thuật toán Gradient Descent.

thư viện còn lại hỗ trợ xử lý dữ liệu

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

Import các thư viện

In [5]:
file_path = '/content/MachineDowntime.csv'  # Đường dẫn tới file
data = pd.read_csv(file_path)

 Đọc dữ liệu từ file CSV và tạo DataFrame data.

In [6]:
print(data.dtypes)

Date                                object
Machine_ID                          object
Assembly_Line_No                    object
Hydraulic_Pressure(bar)            float64
Coolant_Pressure(bar)              float64
Air_System_Pressure(bar)           float64
Coolant_Temperature                float64
Hydraulic_Oil_Temperature(?C)      float64
Spindle_Bearing_Temperature(?C)    float64
Spindle_Vibration(?m)              float64
Tool_Vibration(?m)                 float64
Spindle_Speed(RPM)                 float64
Voltage(volts)                     float64
Torque(Nm)                         float64
Cutting(kN)                        float64
Downtime                            object
dtype: object


Kiểm tra kiểu dữ liệu của từng cột trong DataFrame để biết loại dữ liệu cần xử lý.

In [ ]:
target_column = 'Spindle_Speed(RPM)'

In [ ]:
data[target_column] = pd.to_numeric(data[target_column], errors='coerce')

Chuyển cột mục tiêu sang kiểu số, nếu không chuyển được thì sẽ gán giá trị

In [ ]:
data = data.dropna(subset=[target_column])

Loại bỏ các hàng có giá trị thiếu trong cột mục tiêu để đảm bảo dữ liệu đầu vào hợp lệ.

In [ ]:
y = data[target_column]

In [ ]:
X = data[['Hydraulic_Pressure(bar)', 'Coolant_Pressure(bar)', 'Air_System_Pressure(bar)',
           'Coolant_Temperature', 'Hydraulic_Oil_Temperature(?C)',
           'Spindle_Bearing_Temperature(?C)', 'Spindle_Vibration(?m)',
           'Tool_Vibration(?m)', 'Voltage(volts)', 'Torque(Nm)', 'Cutting(kN)']]



Tập hợp các cột số làm đầu vào để dự đoán.

In [ ]:
if len(y) == 0:
    print("Không có mẫu nào trong cột mục tiêu. Vui lòng kiểm tra dữ liệu.")
    exit()

In [ ]:
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

SimpleImputer: Điền các giá trị thiếu trong tập đầu vào bằng giá trị trung bình của từng cột.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

Chia dữ liệu thành tập huấn luyện (80%) và tập kiểm tra (20%).

In [ ]:
model = SGDRegressor(max_iter=1000, tol=1e-3)
model.fit(X_train, y_train)

SGDRegressor()

Khởi tạo mô hình SGDRegressor và huấn luyện trên tập huấn luyện với số vòng lặp tối đa là 1000 và điều kiện dừng là tol=1e-3.

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 4.728143684898433e+26
R^2 Score: -3.452503720925163e+19


Dự đoán trên tập kiểm tra và tính các chỉ số đánh giá:  

mean_squared_error (MSE): Đo lường mức độ sai số giữa giá trị dự đoán và giá trị thực tế.

r2_score (R²): Đánh giá mức độ phù hợp của mô hình; giá trị gần 1 cho thấy mô hình tốt.